# Computación Cuántica

## Preliminares 
Este material presenta una introducción a la algorítmica básica en el cómputo cuántico, así como conceptos periféricos, utilizando la interfaz de programación contenida en el paquete `Yao` en Julia.

La computacón cuántica es un área mucho más extensa que solo lo que se discutirá aquí, e incluso actualmente no se tiene un panorama consensuado de lo que deberían ser las **capas, arquitectura o flujo de la computación cuántica**, comenzando desde su implementación física hasta cualquier aplicación que haga uso abstracto de dicha implementación.

No obstante, para ejemplificar una posible visión, mostramos la siguiente imagen:

![Capas_QC](https://upload.wikimedia.org/wikipedia/commons/f/fa/Jones-2012-figure01.png)

Aquí podemos apreciar que la algorítmica es una abstracción de las posibles implementaciones físicas que pueden existir y además es resultado de un conjunto de asunciones teóricas que se deben procurar garantizar en las implementaciones mediante cuidados de la capa 2: Virtual (corrección de error, desacoplamiento dinámico, ciclo de control-retroalimentación del sistema, etc.)


## Algorítmica
### Diseño de un circuito
En la algorítmica del cómputo cuántico trabajamos con los siguientes objetos abstractos (que pueden ser implementados de diversas maneras físicamente):
- Registros de qubits
- compuertas lógicas cuántica
- Aparatos de medición 
- Ligaduras de control

Para proveer un ejemplo de un diagrama que contenga todos estos elementos considere el siguiente:

![diagrama_circuito](https://tutorials.yaoquantum.org/dev/generated/quick-start/1.prepare-ghz-state/assets/ghz4.png)

Tenemos un **registro de qubits** que consiste de los qubits $\{Q_0, Q_1, Q_2, Q_3\}$. Éstos están todos en el estado $|0\rangle$, como suele ser común para muchos algoritmos cuánticos.

Los cuadrados, etiquetado uno con una X y otros 7 con H, son ejemplos de compuertas lógicas. Respectivamente se les conoce como compuerta X de Pauli y compuerta de Hadamard.

Éstas, matemáticamente hablando, son operadores unitarios de los cuales, como es bien conocido, tenemos infinitos para cualquier espacio de Hilbert no trivial con el que trabajemos en nuestro sistema cuántico. 

No obstante, elegimos darle nombres a éstas por permitir cálculos limpios en ciertas elecciones de base importantes que podemos realizar en el espacio de Hilbert. Las lineas verticales que conectan algunos de los qubits y poseen un nodo con $\oplus$ y otro relleno son otro ejemplo de compuerta lógica, aunque más compleja

Éstas son compuertas de control que pueden influir en el estado de un qubit en base al estado de otro. Estas son un ejemplo de lo que llamamos **compuertas compuestas** que, matemáticamente, son operadores del espacio producto de los espacios de estado de los qubits involucrados.

Finalmente, al final de la linea de acción de cada uno de los qubits tenemos un elemento de diagrama que simboliza un aparato de medición. Éstos son necesarios para obtener la información contenida en los estados del registro, considerado el **output** o **salida** del algoritmo cuántico que este circuito representa.

### Registro de estados
#### Registros completamente activos

Comencemos creando algunos registros. Primero cargamos el paquete `Yao`:


In [2]:
using Pkg; Pkg.activate("."); using Yao


Podemos crear el registro del diagrama de arriba utilizando la función `zero_state` que nos otorga con facilidad este común registro de cualquier tamaño:


In [3]:
zero_state(4)


Como la naturaleza de la medición que obtenemos de los qubits es inevitablemente probabilística, a veces es deseado tener un número de **batches** o **lotes** que representan varias realizaciones del mismo circuito para obtener estadísticas.

En este siguiente comando creamos 5 lotes de un registro de 3 qubits, todos en el estado $|0\rangle$.


In [4]:
zero_state(3, nbatch = 5)


Podemos también construir un registro a partir de una cadena de bits que representa en análogo clásico del estado cuántico (respecto a alguna base):


In [5]:
product_state(bit"101")


Para cualquier registro podemos extraer el estado explícito como arreglo de números complejos de la siguiente manera:


In [6]:
product_state(bit"101").state



Notemos que este vector no tendrá $3$ elementos correspondiendo a los 3 digitos de la cadena de bits, si no que tendremos $8 = 2^3$.

Por supuesto podemos también construir más de un lote, y también cambiar el tipo de número complejo utilizando para guardar en memoria el estado:


In [7]:
product_state(ComplexF32, bit"111", nbatch = 3)


In [8]:
product_state(ComplexF32, bit"111", nbatch = 3).state


A veces no necesitamos especificar el estado explícito, y queremos en su lugar generar un estado aleatorio que nos permita poner a prueba cierto circuito independiente de sus entradas:


In [9]:
rand_state(2)


In [10]:
 rand_state(2).state ## Estado de 2 qubits tiene 2^2 = 4 entradas.


Otro registro comúnmente utilizado es el registro de estados uniformes:


In [11]:
uniform_state(4).state


Estos tienen la forma: $\frac{1}{n} \sum_k |k\rangle$ y pueden ser similarmente obtenidos tras aplicar una compuerta de hadamar multidimensional a un registro de puros estados $|0\rangle$ (es decir, una compuerta de hadamard *sencilla* por cada qubit). 

Finalmente, mostramos que cualquier registro puede ser representado en su forma hiperbólica, es decir, como elemento de un espacio tensorial general. 



In [12]:
hypercubic(uniform_state(4))


Comparando dimensiones:


In [13]:
uniform_state(4) |> state |> size


In [14]:
 hypercubic(uniform_state(4)) |> size


El último muestra cómo el elemento del espacio vectorial de dimension 16 se entiende como un elemento de un espacio vectorial generado a partir de productos tensoriales de 4 espacios vectoriales de dimensión 2 (es decir, los espacios asociados a cada qubit individual).


#### Registros enfocados
En la práctica, no todos los qubits presentes en el sistema que consideramos el 'computador cuántico' son utilizados para realizar operaciones. Muchos están presentes para asistir a procedimientos de corrección de error o flujo de control entre componentes, pero éstos no son menos importantes de preservar bajo coherencia cuántica.

Estos qubits de ambiente, no obstante, deben ser considerados explícitamente ya que afectan el estado general del registro y `Yao` provee una interfaz para hacer eso. Considere el siguiente registro


In [15]:
zero_state(5) |> focus!(1,4,5)


Notemos que el resultado nos muestra solmente $3$ de los $5$ qubits como qubits activo. Estos qubits activos son lo que conocemos como **qubits de sistema** y son sobre lso que operaremos. El resto solo funcionan como ambiente. El estado general del registro sigue siendo el mismo que si todos fueran activos:


In [16]:
(zero_state(5) |> focus!(1,4,5)).state


Pero en cualquier momento somos capaces de encontrar el número de qubits de sistema y qubits de ambiente de un registro dado:


In [17]:
ψ₁ = zero_state(5) |> focus!(1,4,5)


In [18]:
nqubits(ψ₁), nactive(ψ₁), nremain(ψ₁)


Si queremos '*relajar*' los qubits de ambiente hacia un estado de control con el cual podamos operar, se puede utilizar el comando `relax!` de las siguientes formas:


In [19]:
ψ₁ |> relax!()


In [20]:
relax!(ψ₁)


In [21]:
relax!(ψ₁, to_nactive = 4)


En el último llamado se muestra que podemos controlar el número de qubits del sistema con el que queremos terminar. Note que no elegimos los índices de 'cuales' qubits activar (recordando que antes estaban activos los de índice `5`, `3`, y `2`).

Esto es porque el índice de cierto qubit es un ordemamiento arbitrario: No importa si consideramos que el registro activo consiste de los qubits que originalmente llamamos $\{5,2,3\}$ o de otros, pues esas etiquetas originales fueron arbitrarias para la física.

#### Aritmética de registros
Podemos operar con la aritmética esperada sobre nuestros registros mientras sea teóricamente válida la operación. 


In [22]:
begin
	ψ₂ = rand_state(5)
	ψ₃ = rand_state(5) 
end


In [23]:
(0.3ψ₂ + 2ψ₃)/2


In [24]:
(0.3ψ₂ + 2ψ₃)/2 |> state


No obstante..


In [25]:
(0.3ψ₂ + 2ψ₃)/2 |> isnormalized


Notamos que este nuevo estado no se normaliza por defecto. La **Normalización** es una condición requerida para los estados puros del sistema finito dimensional (como es nuestro caso. En casos más generales de sistemas cuánticos, esta condición se traduce a que el mapa lineal de densidad sea tipo traza con traza igual a 1).

Es decir, requerimos que la norma de la representación pura del estado (aquí denotados como $\psi$) sea igual a 1. Debemos siempre recordar que posterior a alguna operación, debemos normalizar de nuevo el vector representante del estado.


In [26]:
ψ₄ = (0.3ψ₂ + 2ψ₃)/2


In [27]:
ψ₄ |> normalize! |> isnormalized


Exploramos ahora la norma:


In [28]:
ψ₄' * ψ₄ ## la comilla ' representa la operación de transposición en la base selecta.


### Compuertas lógicas y mediciones
Las compuertas lógicas son la forma en que podemos transformar los estados de los registros de qubits que hemos creados. En Yao estos se pueden definir, al igual que los aparatos de medición, en los denominados **bloques**.

Primero, preparemos un registro con la suma de dos registros y lo normalizamos posteriormente:


In [29]:
r = normalize!(ArrayReg(bit"000") + ArrayReg(bit"111"))


In [30]:
r.state


Medir este registro es tan sencillo como utilizar el operador pipe, 'pasándole' el registro al bloque de medición, en este caso, de tres qubits.


In [31]:
r |> Measure(3)


Recordemos que en mecánica cuántica, el acto de permitir que el sistema cuántico aislado interactue con un ambiente de baja coherencia resulta en perder la superposición que antes teníamos y obtener un solo estado bien definido.

Los detalles de cómo sucede esto es, en general, aun un problema abierto conocido como el [problema de medición](https://en.wikipedia.org/wiki/Measurement_problem), siendo aun uno de los misterios más importantes a resolver en los fundamentos teóricos de la mecánica cuántica


In [32]:
r.state


Notamos que el estado mostrado ha colapsado de ser uan superposición a representar un estado con análogo clásico que se puede entender como un resultado definido.

Esto fue un ejemplo de un **bloque**. Los bloques consisten de operaciones generales como medición o compuertas lógicas, pero igualmente Yao nos permite realizar el proceso de medición sin tener que utilizar bloques. La mayoría de transformaciones tienen su versión fuera de bloque:


In [33]:
## Mostrará un resultado distinto si se sigue midiendo...
(product_state(5, 0b10) + product_state(5, 0b11)) |> measure


Las compuertas, fuera de bloques, son representados por sus símbolos comunes para las más utilizadas:


In [34]:
typeof((X, Z, Y, H))


No obstante, nos enfocaremos en el uso dentro de bloques. Para hacer uso de ellos en bloques, debemos especificar cuántas y cuáles compuertas estarán dentro del bloque dado. Utilizamos `repeat` para repetir una compuerta `n` veces.

Como ejemplo, considere el siguiente estado:


In [35]:
ArrayReg(bit"0").state


In [36]:
(ArrayReg(bit"0") |> repeat(1, X)).state


La compuerta $X$ de pauli se conoce como el **bit flip**. Para una lista más detallada y efecto de las compuertas más utlizadas ver [aquí](https://en.wikipedia.org/wiki/Quantum_logic_gate).
